# Ontology Builder

A helpful tool-box to help extend the MIAPPE and SSN/SOSA ontologies with (1) new terms, (2) new relations between terms, and (3) new axioms to define terms.



I think the most complex part of this ontology is the integration of DataFiles and their content (without reading the files at this point).

# Change 
    <!-- http://purl.org/ppeo/PPEO.owl#hasDescription -->

    <owl:DatatypeProperty rdf:about="http://purl.org/ppeo/PPEO.owl#hasDescription">
        <rdfs:range rdf:resource="http://www.w3.org/2001/XMLSchema#string"/>
        <rdfs:label xml:lang="en">has description</rdfs:label>
    </owl:DatatypeProperty>

